# CS6910 Assignment 3 (RNN Frameworks for transliteration) - Without attention

In [1]:
# importing required libraries for the notebook
import pytorch_lightning as plt
from pytorch_lightning.utilities.model_summary import ModelSummary
import torch
import torch.nn as nn
from IPython.display import display
import numpy as np
from torch.utils.data import DataLoader, Dataset
import random
from language import *
from dataset_dataloader import *
from encoder_decoder import *

In [2]:
# know the accelerator available - NOT USED as we have switched to lightning
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# some libraries for plotting model graph
! pip install torchviz graphviz

Defaulting to user installation because normal site-packages is not writeable


# Defining the source and target languages and loading data

In [4]:
# define the source and target languages
TARGET = 'tam'
SOURCE = 'eng'

In [5]:
# load all the available data and print sample counts for each set
x_train, y_train = load_data(TARGET, 'train')
x_valid, y_valid = load_data(TARGET, 'valid')
x_test, y_test = load_data(TARGET, 'test')

print(f'Number of train samples = {len(x_train)}')
print(f'Number of valid samples = {len(x_valid)}')
print(f'Number of test samples = {len(x_test)}')

Number of train samples = 51200
Number of valid samples = 4096
Number of test samples = 4096


In [6]:
# create language objects for storing vocabulary, index2sym and sym2index
SRC_LANG = Language(SOURCE)
TAR_LANG = Language(TARGET)

# creating vocabulary using train data only
SRC_LANG.create_vocabulary(*(x_train))
TAR_LANG.create_vocabulary(*(y_train))

# generate mappings from characters to numbers and vice versa
SRC_LANG.generate_mappings()
TAR_LANG.generate_mappings()

# print the source and target vocabularies
print(f'Source Vocabulary Size = {len(SRC_LANG.symbols)}')
print(f'Source Vocabulary = {SRC_LANG.symbols}')
print(f'Source Mapping {SRC_LANG.index2sym}')
print(f'Target Vocabulary Size = {len(TAR_LANG.symbols)}')
print(f'Target Vocabulary = {TAR_LANG.symbols}')
print(f'Target Mapping {TAR_LANG.index2sym}')

Source Vocabulary Size = 26
Source Vocabulary = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Source Mapping {0: '@', 1: '$', 2: '!', 3: '%', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'j', 14: 'k', 15: 'l', 16: 'm', 17: 'n', 18: 'o', 19: 'p', 20: 'q', 21: 'r', 22: 's', 23: 't', 24: 'u', 25: 'v', 26: 'w', 27: 'x', 28: 'y', 29: 'z'}
Target Vocabulary Size = 46
Target Vocabulary = ['ஃ', 'அ', 'ஆ', 'இ', 'ஈ', 'உ', 'ஊ', 'எ', 'ஏ', 'ஐ', 'ஒ', 'ஓ', 'க', 'ங', 'ச', 'ஜ', 'ஞ', 'ட', 'ண', 'த', 'ந', 'ன', 'ப', 'ம', 'ய', 'ர', 'ற', 'ல', 'ள', 'ழ', 'வ', 'ஷ', 'ஸ', 'ஹ', 'ா', 'ி', 'ீ', 'ு', 'ூ', 'ெ', 'ே', 'ை', 'ொ', 'ோ', 'ௌ', '்']
Target Mapping {0: '@', 1: '$', 2: '!', 3: '%', 4: 'ஃ', 5: 'அ', 6: 'ஆ', 7: 'இ', 8: 'ஈ', 9: 'உ', 10: 'ஊ', 11: 'எ', 12: 'ஏ', 13: 'ஐ', 14: 'ஒ', 15: 'ஓ', 16: 'க', 17: 'ங', 18: 'ச', 19: 'ஜ', 20: 'ஞ', 21: 'ட', 22: 'ண', 23: 'த', 24: 'ந', 25: 'ன', 26: 'ப', 27: 'ம', 28: 'ய',

## Runner Class

In [7]:
class Runner(plt.LightningModule):
    def __init__(self, src_lang : Language, tar_lang : Language, common_embed_size, common_num_layers, 
                 common_hidden_size, common_cell_type, enc_bidirect=False, attention=False, dropout=0.0, 
                 opt_name='Adam', learning_rate=1e-3, batch_size=32):
    
        super(Runner,self).__init__()
        # save the language objects
        self.src_lang = src_lang
        self.tar_lang = tar_lang

        # create all the sub-networks and the main model
        self.encoder = EncoderNet(vocab_size=src_lang.get_size(), embed_size=common_embed_size,
                             num_layers=common_num_layers, hid_size=common_hidden_size,
                             cell_type=common_cell_type, bidirect=enc_bidirect, dropout=dropout)
        if attention:
            self.attention = True
            self.attn_layer = Attention(common_hidden_size, enc_bidirect)
        else:
            self.attention = False
            self.attn_layer = None
        
        self.decoder = DecoderNet(vocab_size=tar_lang.get_size(), embed_size=common_embed_size,
                             num_layers=common_num_layers, hid_size=common_hidden_size,
                             cell_type=common_cell_type, attention=attention, attn_layer=self.attn_layer,
                             enc_bidirect=enc_bidirect, dropout=dropout)
        
        self.model = EncoderDecoder(encoder=self.encoder, decoder=self.decoder, src_lang=src_lang, 
                                    tar_lang=tar_lang)

        # for determinism
        torch.manual_seed(42); torch.cuda.manual_seed(42); np.random.seed(42); random.seed(42)

        self.model.apply(self.init_weights) # initialize model weights
        self.batch_size = batch_size

        # optimizer for the model and loss function [that ignores locs where target = PAD token]
        self.loss_criterion = nn.CrossEntropyLoss(ignore_index=tar_lang.sym2index[PAD_SYM])
        self.opt_name = opt_name
        self.learning_rate = learning_rate

        # only adam is present in configure_optimizers as of now
        if (opt_name != 'Adam'):
            exit(-1)
        
        self.pred_train_words = []
        self.true_train_words = []
        self.pred_valid_words = []
        self.true_valid_words = []

    def configure_optimizers(self):
        optimizer = None
        if self.opt_name == 'Adam':
            optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        return optimizer

    @staticmethod
    def init_weights(m):
        '''
        function to initialize the weights of the model parameters
        '''
        for name, param in m.named_parameters():
            if 'weight' in name:
                 nn.init.uniform_(param.data, -0.04, 0.04)
            else:
                nn.init.constant_(param.data, 0)
    
    @staticmethod
    def exact_accuracy(pred_words, tar_words):
        ''' 
        compute the accuracy using (predicted words, target words) and return it.
        exact word matching is used.
        '''
        assert(len(pred_words) == len(tar_words))
        count = 0
        for i in range(len(pred_words)):
            if pred_words[i] == tar_words[i]:
                count += 1
        return count / len(pred_words)
    
    ####################
    # DATA RELATED HOOKS
    ####################

    def setup(self, stage=None):
        # load all the available data on all GPUs
        self.x_train, self.y_train = load_data(TARGET, 'train')
        self.x_valid, self.y_valid = load_data(TARGET, 'valid')
        self.x_test, self.y_test = load_data(TARGET, 'test')

    def train_dataloader(self):
        dataset = TransliterateDataset(self.x_train, self.y_train, src_lang=SRC_LANG, tar_lang=TAR_LANG)
        dataloader = DataLoader(dataset=dataset, batch_size=self.batch_size, collate_fn=CollationFunction(SRC_LANG, TAR_LANG))
        return dataloader

    def val_dataloader(self):
        dataset = TransliterateDataset(self.x_valid, self.y_valid, src_lang=SRC_LANG, tar_lang=TAR_LANG)
        dataloader = DataLoader(dataset=dataset, batch_size=self.batch_size, collate_fn=CollationFunction(SRC_LANG, TAR_LANG))
        return dataloader

    def test_dataloader(self):
        dataset = TransliterateDataset(self.x_test, self.y_test, src_lang=SRC_LANG, tar_lang=TAR_LANG)
        dataloader = DataLoader(dataset=dataset, batch_size=1, collate_fn=CollationFunction(SRC_LANG, TAR_LANG))
        # we do inference word by word. So, batch_size = 1
        return dataloader

    ####################
    # INTERFACE RELATED FUNCTIONS - NOTE -> add test methods; early stopping; heatmap; beam decoding;
    #                                       save to file and plot test errors (for final); fix teacher forcing time interval;
    #                                       wandb sweeping stuff and model checkpointing;
    #                                       OPT - bleu score/leveshtein distance, model diagram plot
    ####################

    def training_step(self, train_batch, batch_idx):
        batch_X, batch_y, X_lens = train_batch
        # get the logits, preds for the current batch
        logits, preds = self.model(batch_X, batch_y, X_lens, tf_ratio=0.8)
        # ignore loss for the first time step
        targets = batch_y[:, 1:]; logits = logits[:, 1:, :]
        logits = logits.swapaxes(1, 2) # make class logits the second dimension as needed
        loss = self.loss_criterion(logits, targets)
        # for epoch-level metrics[accuracy], log all the required data
        self.true_train_words += self.tar_lang.convert_to_words(batch_y)
        self.pred_train_words += self.tar_lang.convert_to_words(preds)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def on_train_epoch_end(self):
        self.log('train_acc', 100*self.exact_accuracy(self.pred_train_words, self.true_train_words), 
                 on_epoch=True, prog_bar=True)
        self.pred_train_words.clear(); self.true_train_words.clear()

    def validation_step(self, valid_batch, batch_idx):
        batch_X, batch_y, X_lens = valid_batch
        # get the logits, preds for the current batch
        logits, preds = self.model(batch_X, batch_y, X_lens) # no teacher forcing
        # ignore loss for the first time step
        targets = batch_y[:, 1:]; logits = logits[:, 1:, :]
        logits = logits.swapaxes(1, 2) # make class logits the second dimension as needed
        loss = self.loss_criterion(logits, targets)
        # for epoch-level metrics[accuracy], log all the required data
        self.true_valid_words += self.tar_lang.convert_to_words(batch_y)
        self.pred_valid_words += self.tar_lang.convert_to_words(preds)
        self.log('valid_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
    
    def on_validation_epoch_end(self):
        self.log('valid_acc', 100*self.exact_accuracy(self.true_valid_words, self.pred_valid_words), 
                 on_epoch=True, prog_bar=True)
        self.true_valid_words.clear(); self.pred_valid_words.clear()

In [8]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

# BEST CONFIG - NOTE - NOTE
runner = Runner(SRC_LANG, TAR_LANG, 128, 3, 256, 'LSTM', True, False, 0.0, 'Adam', learning_rate=2e-3, batch_size=128)
#trainer = plt.Trainer(max_epochs=20)
#trainer.fit(runner)

model_sum = ModelSummary(runner)
print(model_sum)

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 4.3 M 
1 | decoder        | DecoderNet       | 1.5 M 
2 | model          | EncoderDecoder   | 5.8 M 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
5.8 M     Trainable params
0         Non-trainable params
5.8 M     Total params
23.238    Total estimated model params size (MB)


In [15]:
# NOTE -> performing BUG SEARCH
# testing all combinations to catch bugs
# RESULT -> all clear; no bugs caught
num_lay = [1,3]
ctype = ['LSTM', 'GRU', 'RNN']
bidirect = [True, False]
attn = [True, False]

for n in num_lay:
    for c in ctype:
        for b in bidirect:
            for a in attn:
                runner = Runner(SRC_LANG, TAR_LANG, 128, n, 256, c, b, a, 0.05, 'Adam', learning_rate=2e-3, batch_size=128)
                trainer = plt.Trainer(max_steps=50)
                trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 925 K 
1 | attn_layer     | Attention        | 197 K 
2 | decoder        | DecoderNet       | 1.1 M 
3 | model          | EncoderDecoder   | 2.1 M 
4 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.246     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/vikram/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/vikram/.local/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 925 K 
1 | decoder        | DecoderNet       | 414 K 
2 | model          | EncoderDecoder   | 1.3 M 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.361     Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=50` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 365 K 
1 | decoder        | DecoderNet       | 381 K 
2 | model          | EncoderDecoder   | 747 K 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
747 K     Trainable params
0         Non-trainable params
747 K     Total params
2.988     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_steps=50` reached.


In [17]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

runner = Runner(SRC_LANG, TAR_LANG, 32, 1, 128, 'GRU', True, True, 0.2, 'Adam', learning_rate=1e-3, batch_size=128)
trainer = lt.Trainer(max_epochs=1)
trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 158 K 
1 | attn_layer     | Attention        | 49.4 K
2 | decoder        | DecoderNet       | 220 K 
3 | model          | EncoderDecoder   | 379 K 
4 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
379 K     Trainable params
0         Non-trainable params
379 K     Total params
1.517     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


51200
('फिोऑऋचउ्म्ीद़उंंं', 'शस्त्रागार')
('झफञैैृऋसऔदंंंंंंं', 'बिन्द्या')
('ढढथथथ', 'किरणकांत')
('थगअच', 'यज्ञोपवीत')
('फििीघघैैफंचंटऽचचं', 'रटानिया')
('ढि्मँऽ', 'वागण्याचे')
('ढचइॉेऋऋआआऽऋढशढढॉच', 'देशभरामध्ये')
('फिञैैजजैऑःटटचचचउच', 'सुघड़पन')
('फििफणङऋुहहंचछवंंं', 'मोहीवाल')
('ौँौऽषऑंतऽ', 'सर्वसंग्रह')
('थगोससफऑंंचचचचचचछच', 'बसेको')
('िथञँँऽौधअःससढढचछं', 'तुमच्यापैकी')
('ेः््ऽससऑफऽअफिॉचछच', 'कान्यकुब्ज')
('थगिघऊ', 'इनटॉक्सिनेशन')
('गतततःसससञॉघफिथऽवं', 'मेच्यूरिटी')
('थगङणउफिंचंंंंंंंं', 'अगरी')
('', 'अनुक्रमानुपात')
('', 'धूलचन्द')
('फिाटचंऑटंंंंंटऽवट', 'अवलेह')
('ऽझफसङण्ंफचंचचचचउच', 'एबरोर्ड')
('फिःटउछषषंचंटचचंंट', 'बैलर्स')
('थगअणऔँँ%थंचचछंचछच', 'बार्गली')
('फिटोफ', 'पंक्चर्ड')
('ढइथढथौटटइचचचचछंंं', 'हैंकर्स')
('फि@औआऑग', 'जवानोंके')
('ढॉ%ऽ', 'पौंकी')
('फधँटम्षषंचचचचउंंं', 'जगनधाम')
('झँििऽघैसँफचचच%चछच', 'पोन्नियम')
('तफणेइ्चचफचचचछआचचं', 'आईएनबीए')
('फि्िथँँचचचचंटंंटं', 'वेदलम')
('फिञिञीघइलचचचछंकंट', 'चिनवास')
('फि्णबऔीज्फंंंंंटं', 'मारवाड़ा')
('फिगङअफगफंंंंंंंंं', 'अनसमझा')
('ढि

In [13]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

runner = Runner(SRC_LANG, TAR_LANG, 32, 3, 128, 'LSTM', True, False, 0.2, 'Adam', learning_rate=1e-3, batch_size=128)
trainer = plt.Trainer(max_epochs=1)
trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 1.1 M 
1 | decoder        | DecoderNet       | 358 K 
2 | model          | EncoderDecoder   | 1.4 M 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.657     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


51200
('स', 'शस्त्रागार')
('न%ररोअणणढव!आआआगआआ', 'बिन्द्या')
('ऐउउउ', 'किरणकांत')
('', 'यज्ञोपवीत')
('', 'रटानिया')
('', 'वागण्याचे')
('', 'देशभरामध्ये')
('ऽऐऐऐअअऐ्््गगआआआआआ', 'सुघड़पन')
('', 'मोहीवाल')
('!ाावव@ममणणणढााृखआ', 'सर्वसंग्रह')
('ढढढऐचऐऐ@डआआआआआआआआ', 'बसेको')
('', 'तुमच्यापैकी')
('', 'कान्यकुब्ज')
('%रररररहऐऐऐऐऐोराफज', 'इनटॉक्सिनेशन')
('नडऐऐऐ', 'मेच्यूरिटी')
('', 'अगरी')
('र', 'अनुक्रमानुपात')
('', 'धूलचन्द')
('', 'अवलेह')
('ग', 'एबरोर्ड')
('ौढ', 'बैलर्स')
('', 'बार्गली')
('', 'पंक्चर्ड')
('ऐऽा', 'हैंकर्स')
('ऽ', 'जवानोंके')
('', 'पौंकी')
('', 'जगनधाम')
('', 'पोन्नियम')
('ऐ%%!उरररधशशआआआआआआ', 'आईएनबीए')
('', 'वेदलम')
('उककडरइइइवववआआआआआऊ', 'चिनवास')
('ऽऊऊऊरररररर!!ववगगआ', 'मारवाड़ा')
('उ%', 'अनसमझा')
('उउस', 'इस्टूडेंट')
('गमऐऽऽढऽववइगगआगगगग', 'दुःखीत')
('ऐ', 'सिघांची')
('लँँाऐ', 'शिक्षेचे')
('चऽँऔऽऔऔझझआआआआआआआऑ', 'उज्वलतम')
('%ऋगगउउफढ@वफऑआऑचआज', 'आपट्टन')
('ऊ', 'अम्बिकावन')
('ऽऽऐाणहैवआआआआआआआआआ', 'खगनी')
('%ऋ%ोककरररागगआगगखज', 'थियामिन')
('बगगडाााााॅसररररतख', 'सोशियोलाजिकल')
('ऽऽउऐऔ

In [14]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

runner = Runner(SRC_LANG, TAR_LANG, 32, 3, 128, 'LSTM', True, False, 0.2, 'Adam', learning_rate=1e-3, batch_size=128)
trainer = plt.Trainer(max_epochs=1)
trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 1.1 M 
1 | decoder        | DecoderNet       | 358 K 
2 | model          | EncoderDecoder   | 1.4 M 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.657     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


51200
('स', 'शस्त्रागार')
('न%ररोअणणढव!आआआगआआ', 'बिन्द्या')
('ऐउउउ', 'किरणकांत')
('', 'यज्ञोपवीत')
('', 'रटानिया')
('', 'वागण्याचे')
('', 'देशभरामध्ये')
('ऽऐऐऐअअऐ्््गगआआआआआ', 'सुघड़पन')
('', 'मोहीवाल')
('!ाावव@ममणणणढााृखआ', 'सर्वसंग्रह')
('ढढढऐचऐऐ@डआआआआआआआआ', 'बसेको')
('', 'तुमच्यापैकी')
('', 'कान्यकुब्ज')
('%रररररहऐऐऐऐऐोराफज', 'इनटॉक्सिनेशन')
('नडऐऐऐ', 'मेच्यूरिटी')
('', 'अगरी')
('र', 'अनुक्रमानुपात')
('', 'धूलचन्द')
('', 'अवलेह')
('ग', 'एबरोर्ड')
('ौढ', 'बैलर्स')
('', 'बार्गली')
('', 'पंक्चर्ड')
('ऐऽा', 'हैंकर्स')
('ऽ', 'जवानोंके')
('', 'पौंकी')
('', 'जगनधाम')
('', 'पोन्नियम')
('ऐ%%!उरररधशशआआआआआआ', 'आईएनबीए')
('', 'वेदलम')
('उककडरइइइवववआआआआआऊ', 'चिनवास')
('ऽऊऊऊरररररर!!ववगगआ', 'मारवाड़ा')
('उ%', 'अनसमझा')
('उउस', 'इस्टूडेंट')
('गमऐऽऽढऽववइगगआगगगग', 'दुःखीत')
('ऐ', 'सिघांची')
('लँँाऐ', 'शिक्षेचे')
('चऽँऔऽऔऔझझआआआआआआआऑ', 'उज्वलतम')
('%ऋगगउउफढ@वफऑआऑचआज', 'आपट्टन')
('ऊ', 'अम्बिकावन')
('ऽऽऐाणहैवआआआआआआआआआ', 'खगनी')
('%ऋ%ोककरररागगआगगखज', 'थियामिन')
('बगगडाााााॅसररररतख', 'सोशियोलाजिकल')
('ऽऽउऐऔ

In [15]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

runner = Runner(SRC_LANG, TAR_LANG, 32, 3, 128, 'GRU', True, False, 0.2, 'Adam', learning_rate=1e-3, batch_size=128)
trainer = plt.Trainer(max_epochs=1)
trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 816 K 
1 | decoder        | DecoderNet       | 271 K 
2 | model          | EncoderDecoder   | 1.1 M 
3 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.353     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


51200
('ळळतततममससतडबँँँओथ', 'शस्त्रागार')
('तौ', 'बिन्द्या')
('ःौौौौटटटटटटटटधञञञ', 'किरणकांत')
('धधौतधममझझझसोोमवव', 'यज्ञोपवीत')
('ंटयटटटटतततसूऔेेञे', 'रटानिया')
('ौततसौछछसफफफसऐऐऐेे', 'वागण्याचे')
('मततततननबबनअबमभफमम', 'देशभरामध्ये')
('मूळळममोंनईृधधधञञञ', 'सुघड़पन')
('दममखखखखतततँँँेोेञ', 'मोहीवाल')
('तौटममखखखमममममोञेे', 'सर्वसंग्रह')
('मम', 'बसेको')
('ळधधचनमममम', 'तुमच्यापैकी')
('ळळआसससससटटङटअअअऊअ', 'कान्यकुब्ज')
('ळळसऐऐऐञञळमममखखखखख', 'इनटॉक्सिनेशन')
('ऽममममखे', 'मेच्यूरिटी')
('गछछछयंयफय%थञञेेेे', 'अगरी')
('तधधछऊअअअ%%%%थथ%%%', 'अनुक्रमानुपात')
('ंटओटटटटखखखधध्ञय्े', 'धूलचन्द')
('धळळयळसतमखेे्ख@', 'अवलेह')
('ँळतऔऔऔऔममम्््््ेे', 'एबरोर्ड')
('ऽौऽआंंंखमञ%थेेञञे', 'बैलर्स')
('एटटटटटटसससरञञञटय', 'बार्गली')
('ळळळंंमळमममखख', 'पंक्चर्ड')
('धधँममखममममखमयय', 'हैंकर्स')
('ंळतततसझटटटटटऐञ%टट', 'जवानोंके')
('ळळळळफततस्ञेे्धधेे', 'पौंकी')
('खटयययबससऐ्ऐपञऊञञञ', 'जगनधाम')
('छळळल', 'पोन्नियम')
('ळँयछँढढसञ्िौौ्ेेे', 'आईएनबीए')
('धधतततससनमथञेेअििम', 'वेदलम')
('ममममखखखममङञञञञथेे', 'चिनवास')
('ळळळ!औऔँँँंंंअथञेे

In [16]:
# testing runner
# keep embedding small (around 32) -> important to get dense embedding
# also, adjust learning rate reasonably

runner = Runner(SRC_LANG, TAR_LANG, 32, 3, 128, 'GRU', True, True, 0.2, 'Adam', learning_rate=1e-3, batch_size=128)
trainer = plt.Trainer(max_epochs=1)
trainer.fit(runner)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type             | Params
----------------------------------------------------
0 | encoder        | EncoderNet       | 816 K 
1 | attn_layer     | Attention        | 49.4 K
2 | decoder        | DecoderNet       | 419 K 
3 | model          | EncoderDecoder   | 1.2 M 
4 | loss_criterion | CrossEntropyLoss | 0     
----------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.944     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


51200
('ऋृृेाुुठऐऐऐऐचढऑऑु', 'शस्त्रागार')
('ऽृृऑऑऑऑऑऔऑऔऑऑुुहु', 'बिन्द्या')
('ृशशशऐऐऐऐऐछढढहऑययळ', 'किरणकांत')
('ृृुुैैैटटटैऑृुङङङ', 'यज्ञोपवीत')
('%ुुुऑपमशममिलढ््््', 'रटानिया')
('ृषशशशईााईईई़ुऑऑऑ्', 'वागण्याचे')
('ृङााऐऐऐऐशशशषढझऑऑु', 'देशभरामध्ये')
('%ुुुटैै@ृृ्््ुुुु', 'सुघड़पन')
('दँृृङऔङऔ@@@ुहहहुऑ', 'मोहीवाल')
('ऋऋाुुुुेेेाआआटट््', 'सर्वसंग्रह')
('आदेऐऐऐ़ुुुुुु््ुि', 'बसेको')
('ंंं%%ऑुऑुङशशऐऐआढ्', 'तुमच्यापैकी')
('ष%ऽऽ!!ओओओओषषुुजुु', 'कान्यकुब्ज')
('ृशङङऊ@शाोुखझझऽऽॅ@', 'इनटॉक्सिनेशन')
('ंओओईईुुछठठ@भुुुुु', 'मेच्यूरिटी')
('षऽऽभभभभ@@@@य@िुिऑ', 'अगरी')
('ऋईऽऽङङआआआआङङ!ृँृि', 'अनुक्रमानुपात')
('ओओँऽझभभभभभऑ्लहह@@', 'धूलचन्द')
('ऽृृळ@@ीऑऑऑुऑिििळ@', 'अवलेह')
('%ृॅजुुशशशढऑऑऑुुुऑ', 'एबरोर्ड')
('ंंंदखढझेुुु्ुुुृु', 'बैलर्स')
('घषवघघघभभऑऑऑऑऑऑुुु', 'बार्गली')
('घघखँखआठठुषषहृिुिआ', 'पंक्चर्ड')
('घुजजटघघुुुुुु्हहह', 'हैंकर्स')
('ैररटऊखटढअढढढऑऑऑऑऑ', 'जवानोंके')
('आाागईईईभजऑऑऑ्िुुु', 'पौंकी')
('ववॅॅॅञञञढआऋऑुििऑऑ', 'जगनधाम')
('%ळषििळ!!मऽढढिऑहहह', 'पोन्नियम')
('ृृुेऐाकककयययिुििु', 'आईएनबीए')
('ैैशशशब